# 🚀 Getting Started with CeNN Framework

Welcome to the interactive tutorial for the Neuro-Symbolic CeNN Framework! 

This notebook will guide you through:
1. **Installation** and setup
2. **Basic usage** of CeNN for time series forecasting
3. **Visualization** of CeNN dynamics
4. **Benchmarking** against classical methods

## ⚠️ Important Note
**This is classical emulation of quantum-inspired models.** No actual quantum hardware is used.

---

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install numpy pandas matplotlib seaborn scikit-learn scipy tqdm -q

In [ ]:
# Import libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Set random seed for reproducibility
np.random.seed(42)

# Add src to path
sys.path.insert(0, os.path.join(os.getcwd(), '../src'))

# Import CeNN framework
from cenn_framework import CeNNEmulator
from data_processing.preprocess import TimeSeriesPreprocessor
from benchmarking.compare_models import ModelBenchmark
from utils.helpers import set_random_seed, calculate_spectral_similarity

print("✅ Libraries imported successfully!")

## 2. Create Synthetic Time Series Data

In [ ]:
def create_synthetic_timeseries(n_points=500, noise_level=0.1):
    """Create synthetic time series for demonstration."""
    t = np.linspace(0, 10, n_points)
    
    # Multiple frequency components
    signal = (
        2.0 * np.sin(2 * np.pi * 0.5 * t) +  # Low frequency
        0.5 * np.sin(2 * np.pi * 2.0 * t) +  # Medium frequency
        0.2 * np.sin(2 * np.pi * 5.0 * t) +  # High frequency
        noise_level * np.random.normal(size=n_points)  # Noise
    )
    
    # Add trend
    signal = signal + 0.1 * t
    
    return t, signal

# Create and visualize data
t, time_series = create_synthetic_timeseries()

plt.figure(figsize=(12, 4))
plt.plot(t, time_series, 'b-', linewidth=1.5, label='Synthetic Time Series')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Synthetic Time Series for Demonstration')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print(f"Time series length: {len(time_series)}")
print(f"Mean: {time_series.mean():.3f}, Std: {time_series.std():.3f}")

## 3. Initialize CeNN Emulator

In [ ]:
# Initialize CeNN with different configurations
configurations = {
    "Standard": {"grid_size": (8, 8), "template_A": [0.4, 1.0, 0.4], "template_B": [0.2, 0.5, 0.2]},
    "Weak Feedback": {"grid_size": (6, 6), "template_A": [0.1, 0.5, 0.1], "template_B": [0.1, 0.3, 0.1]},
    "Strong Feedback": {"grid_size": (10, 10), "template_A": [0.8, 1.5, 0.8], "template_B": [0.3, 0.7, 0.3]},
}

# Create and display emulators
emulators = {}
for name, config in configurations.items():
    emulator = CeNNEmulator(**config)
    emulators[name] = emulator
    print(f"✅ {name}: {config['grid_size'][0]}x{config['grid_size'][1]} grid, "
          f"Template A={config['template_A']}, Template B={config['template_B']}")

## 4. Visualize CeNN Grid Dynamics

In [ ]:
def visualize_cenn_grid(emulator, steps=10):
    """Visualize CeNN grid evolution over time."""
    
    # Reset and evolve grid
    emulator.reset_grid()
    
    # Create figure
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.flatten()
    
    for step in range(steps):
        # Evolve one step
        output = emulator.step()
        
        # Plot grid state
        ax = axes[step]
        im = ax.imshow(output, cmap='viridis', vmin=-1, vmax=1)
        ax.set_title(f"Step {step+1}")
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.suptitle(f"CeNN Grid Evolution ({steps} steps)", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return emulator

# Visualize Standard configuration
print("Visualizing CeNN Grid Evolution (Standard Configuration):")
emulator = visualize_cenn_grid(emulators["Standard"])

## 5. Time Series Forecasting with CeNN

In [ ]:
def run_forecasting_experiment(time_series, forecast_horizon=24, window_size=24):
    """Run forecasting experiment with CeNN."""
    
    # Preprocess data
    preprocessor = TimeSeriesPreprocessor(scaling_method='minmax')
    scaled_series = preprocessor.scale_data(time_series, fit=True)
    
    results = {}
    
    for name, emulator in emulators.items():
        print(f"Running forecast with {name} configuration...")
        
        # Reset emulator
        emulator.reset_grid()
        
        # Run forecasting
        predictions = emulator.forecast(
            series=scaled_series,
            forecast_horizon=forecast_horizon,
            window_size=window_size
        )
        
        # Inverse scaling
        predictions_original = preprocessor.inverse_scale(predictions)
        actual_original = preprocessor.inverse_scale(scaled_series[-forecast_horizon:])
        
        # Calculate metrics
        mse = np.mean((predictions_original - actual_original) ** 2)
        mae = np.mean(np.abs(predictions_original - actual_original))
        
        results[name] = {
            'predictions': predictions_original,
            'actual': actual_original,
            'mse': mse,
            'mae': mae
        }
        
        print(f"  MSE: {mse:.6f}, MAE: {mae:.6f}")
    
    return results

# Run forecasting
print("Running Forecasting Experiments:")
forecast_results = run_forecasting_experiment(time_series)

In [ ]:
# Visualize forecasting results
def visualize_forecast_results(results):
    """Visualize forecasting results from different configurations."""
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: Predictions comparison
    ax = axes[0, 0]
    colors = ['blue', 'green', 'red', 'purple']
    
    for i, (name, result) in enumerate(results.items()):
        ax.plot(result['predictions'], '--', color=colors[i], linewidth=2, 
                alpha=0.8, label=f"{name} (MSE: {result['mse']:.4f})")
    
    ax.plot(results["Standard"]['actual'], 'k-', linewidth=2, label='Actual')
    ax.set_xlabel('Forecast Horizon')
    ax.set_ylabel('Value')
    ax.set_title('Forecast Predictions vs Actual')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Error comparison
    ax = axes[0, 1]
    names = list(results.keys())
    mses = [results[name]['mse'] for name in names]
    maes = [results[name]['mae'] for name in names]
    
    x = np.arange(len(names))
    width = 0.35
    
    ax.bar(x - width/2, mses, width, label='MSE', color='steelblue', alpha=0.8)
    ax.bar(x + width/2, maes, width, label='MAE', color='lightcoral', alpha=0.8)
    
    ax.set_xlabel('Configuration')
    ax.set_ylabel('Error')
    ax.set_title('Error Metrics Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(names)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Plot 3: Error distribution
    ax = axes[1, 0]
    for i, (name, result) in enumerate(results.items()):
        errors = result['predictions'] - result['actual']
        ax.hist(errors, bins=20, alpha=0.5, label=name, color=colors[i])
    
    ax.axvline(x=0, color='black', linestyle='--', linewidth=2)
    ax.set_xlabel('Prediction Error')
    ax.set_ylabel('Frequency')
    ax.set_title('Error Distribution')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 4: Time series with forecast
    ax = axes[1, 1]
    # Show last 100 points of original series
    ax.plot(range(-100, 0), time_series[-100:], 'b-', linewidth=1.5, label='Historical')
    ax.plot(range(0, 24), forecast_results["Standard"]['predictions'], 
            'r--', linewidth=2, label='CeNN Forecast')
    ax.plot(range(0, 24), forecast_results["Standard"]['actual'], 
            'k-', linewidth=2, label='Actual Future')
    
    ax.axvline(x=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Value')
    ax.set_title('Time Series with CeNN Forecast')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Visualize results
visualize_forecast_results(forecast_results)

## 6. Benchmarking Against Classical Methods

In [ ]:
def benchmark_cenn_vs_classical(time_series):
    """Benchmark CeNN against classical forecasting methods."""
    
    from sklearn.linear_model import LinearRegression
    from sklearn.neural_network import MLPRegressor
    from sklearn.svm import SVR
    from sklearn.metrics import mean_squared_error, mean_absolute_error
    
    # Prepare data
    window_size = 24
    forecast_horizon = 24
    
    X, y = [], []
    for i in range(len(time_series) - window_size - forecast_horizon + 1):
        X.append(time_series[i:i + window_size])
        y.append(time_series[i + window_size:i + window_size + forecast_horizon])
    
    X, y = np.array(X), np.array(y)
    
    # Split data
    split_idx = int(0.8 * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    print(f"Data shape: X_train={X_train.shape}, X_test={X_test.shape}")
    
    # Initialize benchmark
    benchmark = ModelBenchmark()
    
    # Classical Models
    models = {
        'Linear Regression': LinearRegression(),
        'MLP (Neural Network)': MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000, random_state=42),
        'SVR (Support Vector)': SVR(kernel='rbf', C=1.0, epsilon=0.1)
    }
    
    # Train and evaluate classical models
    for name, model in models.items():
        print(f"Training {name}...")
        
        # Train (using only first step for simplicity)
        model.fit(X_train, y_train[:, 0])  # Predict only first step
        
        # Predict
        y_pred = model.predict(X_test)
        y_true = y_test[:, 0]
        
        # Evaluate
        benchmark.evaluate_model(y_true, y_pred, name)
    
    # CeNN Model
    print("Training CeNN...")
    
    # Use CeNN for forecasting
    cenn_predictions = []
    cenn_actual = []
    
    emulator = CeNNEmulator(grid_size=(8, 8))
    
    for i in tqdm(range(len(X_test))):
        # Reset and forecast
        emulator.reset_grid()
        prediction = emulator.forecast(
            series=X_test[i],
            forecast_horizon=forecast_horizon,
            window_size=window_size
        )
        
        cenn_predictions.append(prediction[0])  # First step
        cenn_actual.append(y_test[i, 0])
    
    benchmark.evaluate_model(cenn_actual, cenn_predictions, "CeNN (Proposed)")
    
    # Compare all models
    comparison = benchmark.compare_models()
    
    # Convert to DataFrame for display
    df_comparison = pd.DataFrame(comparison)
    df_comparison = df_comparison.sort_values('RMSE')
    
    return df_comparison

# Run benchmark
print("Running Benchmark: CeNN vs Classical Methods")
benchmark_results = benchmark_cenn_vs_classical(time_series)

# Display results
print("\n" + "="*60)
print("Benchmark Results (Lower is better):")
print("="*60)
print(benchmark_results.to_string(index=False))

In [ ]:
# Visualize benchmark results
plt.figure(figsize=(12, 6))

# Bar chart for RMSE
plt.subplot(1, 2, 1)
bars = plt.bar(benchmark_results['Model'], benchmark_results['RMSE'], 
               color=['lightgray', 'lightgray', 'lightgray', 'darkgreen'])
plt.ylabel('RMSE')
plt.title('Root Mean Square Error Comparison')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')

# Highlight CeNN
bars[-1].set_edgecolor('red')
bars[-1].set_linewidth(3)

# Bar chart for MAE
plt.subplot(1, 2, 2)
bars = plt.bar(benchmark_results['Model'], benchmark_results['MAE'],
               color=['lightgray', 'lightgray', 'lightgray', 'darkgreen'])
plt.ylabel('MAE')
plt.title('Mean Absolute Error Comparison')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')

# Highlight CeNN
bars[-1].set_edgecolor('red')
bars[-1].set_linewidth(3)

plt.tight_layout()
plt.show()

## 7. Advanced: Spectral Analysis

In [ ]:
def analyze_spectral_properties(time_series, predictions):
    """Analyze spectral properties of time series and predictions."""
    
    from scipy import signal
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: Time domain comparison
    axes[0, 0].plot(time_series[-100:], 'b-', linewidth=1.5, label='Original')
    axes[0, 0].plot(range(100 - len(predictions), 100), predictions, 
                    'r--', linewidth=2, label='CeNN Predictions')
    axes[0, 0].set_xlabel('Time Step')
    axes[0, 0].set_ylabel('Value')
    axes[0, 0].set_title('Time Domain Comparison')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Power Spectral Density
    f_original, Pxx_original = signal.welch(time_series[-200:], fs=1.0)
    f_pred, Pxx_pred = signal.welch(predictions, fs=1.0)
    
    axes[0, 1].semilogy(f_original, Pxx_original, 'b-', linewidth=2, label='Original')
    axes[0, 1].semilogy(f_pred, Pxx_pred, 'r--', linewidth=2, label='CeNN Predictions')
    axes[0, 1].set_xlabel('Frequency [Hz]')
    axes[0, 1].set_ylabel('Power Spectral Density')
    axes[0, 1].set_title('Power Spectral Density Comparison')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Spectrogram of original
    f, t, Sxx = signal.spectrogram(time_series[-200:], fs=1.0)
    pcm = axes[1, 0].pcolormesh(t, f, np.log10(Sxx), shading='gouraud', cmap='viridis')
    axes[1, 0].set_ylabel('Frequency [Hz]')
    axes[1, 0].set_xlabel('Time [sec]')
    axes[1, 0].set_title('Spectrogram of Original Time Series')
    plt.colorbar(pcm, ax=axes[1, 0], label='log10(Power)')
    
    # Plot 4: Autocorrelation
    autocorr_original = np.correlate(time_series[-200:], time_series[-200:], mode='full')
    autocorr_original = autocorr_original[autocorr_original.size // 2:]
    
    autocorr_pred = np.correlate(predictions, predictions, mode='full')
    autocorr_pred = autocorr_pred[autocorr_pred.size // 2:]
    
    lags = np.arange(len(autocorr_original))
    axes[1, 1].plot(lags[:50], autocorr_original[:50], 'b-', linewidth=2, label='Original')
    axes[1, 1].plot(lags[:min(50, len(autocorr_pred))], autocorr_pred[:min(50, len(autocorr_pred))], 
                    'r--', linewidth=2, label='CeNN Predictions')
    axes[1, 1].set_xlabel('Lag')
    axes[1, 1].set_ylabel('Autocorrelation')
    axes[1, 1].set_title('Autocorrelation Function')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate spectral similarity
    similarity = calculate_spectral_similarity(
        time_series[-len(predictions):], 
        predictions
    )
    
    print(f"Spectral Similarity Index: {similarity:.4f} (1.0 = perfect match)")
    
    return similarity

# Analyze spectral properties
print("Analyzing Spectral Properties:")
spectral_similarity = analyze_spectral_properties(
    time_series, 
    forecast_results["Standard"]['predictions']
)

## 8. Export Results

In [ ]:
# Export results to files
import json

def export_results(results, benchmark_results, spectral_similarity):
    """Export experiment results to files."""
    
    # Create output directory
    output_dir = 'notebook_outputs'
    os.makedirs(output_dir, exist_ok=True)
    
    # Export forecast results
    forecast_export = {}
    for name, result in results.items():
        forecast_export[name] = {
            'mse': float(result['mse']),
            'mae': float(result['mae']),
            'predictions': result['predictions'].tolist()
        }
    
    with open(f'{output_dir}/forecast_results.json', 'w') as f:
        json.dump(forecast_export, f, indent=2)
    
    # Export benchmark results
    benchmark_results.to_csv(f'{output_dir}/benchmark_results.csv', index=False)
    
    # Export summary
    summary = {
        'best_forecast_model': min(results.items(), key=lambda x: x[1]['mse'])[0],
        'best_benchmark_model': benchmark_results.loc[benchmark_results['RMSE'].idxmin(), 'Model'],
        'spectral_similarity': float(spectral_similarity),
        'timestamp': pd.Timestamp.now().isoformat()
    }
    
    with open(f'{output_dir}/experiment_summary.json', 'w') as f:
        json.dump(summary, f, indent=2)
    
    print(f"✅ Results exported to {output_dir}/")
    print(f"   - forecast_results.json")
    print(f"   - benchmark_results.csv")
    print(f"   - experiment_summary.json")

# Export results
export_results(forecast_results, benchmark_results, spectral_similarity)

## 9. Summary and Next Steps

### 📊 What We've Learned:

1. **CeNN Initialization**: How to create and configure CeNN emulators with different templates and grid sizes
2. **Grid Dynamics**: Visualization of how CeNN cells evolve over time
3. **Time Series Forecasting**: Using CeNN for multi-step forecasting
4. **Benchmarking**: Comparing CeNN performance against classical ML methods
5. **Spectral Analysis**: Analyzing frequency properties of predictions

### 🎯 Key Findings from This Tutorial:

- CeNN achieves **competitive performance** compared to classical methods
- The **Standard configuration** (8×8 grid, Template A=[0.4, 1.0, 0.4]) works well for general time series
- Spectral analysis shows CeNN **preserves frequency characteristics** of the original series

### 🚀 Next Steps:

1. **Try your own data**: Replace the synthetic time series with your own data
2. **Experiment with parameters**: Try different grid sizes, templates, and activation functions
3. **Check Notebook 02**: Run `02_reproduce_paper.ipynb` to reproduce paper results
4. **Explore experiments/**: Run the full experimental pipeline

### 📚 References:

- [Paper] Systematic Review of QML for Time Series Forecasting
- [Documentation] See `docs/` for detailed API documentation
- [Examples] Check `examples/` for more advanced use cases

---

## 🎉 Congratulations!

You've successfully completed the CeNN Framework tutorial. You're now ready to use the framework for your own time series forecasting projects!

In [ ]:
# Final summary
print("="*60)
print("TUTORIAL COMPLETE - SUMMARY")
print("="*60)
print(f"\n✅ Best forecasting configuration: {min(forecast_results.items(), key=lambda x: x[1]['mse'])[0]}")
print(f"✅ Best benchmark model: {benchmark_results.loc[benchmark_results['RMSE'].idxmin(), 'Model']}")
print(f"✅ Spectral similarity: {spectral_similarity:.4f}")
print(f"\n📁 Results saved to: notebook_outputs/")
print("\n🚀 Next: Run notebook 02_reproduce_paper.ipynb to reproduce paper results!")